In [1]:
import polars as pl  # Better than pandas
from SPARQLWrapper import SPARQLWrapper, JSON as JsonReturn
from tqdm import tqdm

In [2]:
# Display longer strings
pl.Config.set_fmt_str_lengths(100)

polars.config.Config

In [3]:
class WikidataRequests:
    def __init__(self) -> None:
        self.sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        self.sparql.setReturnFormat(JsonReturn)

    def query(self, query: str, schema=None):
        self.sparql.setQuery(query)
        result: dict = self.sparql.queryAndConvert()  # type:ignore

        return pl.DataFrame(result["results"]["bindings"], schema=schema).select(
            pl.col("*").apply(lambda var: var["value"])
        )

    def find_country(self, name: str):
        return self.query(
            f"""\
SELECT (?country as ?uri) WHERE {{
    ?country rdfs:label "{name}"@en;
        wdt:P31 wd:Q3624078. # 'instance of' 'sovereign state'
    FILTER(NOT EXISTS {{ ?country (p:P31/ps:P31) wd:Q3024240. }}) # is NOT a 'historical country'
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}
LIMIT 1""",
            schema=["uri"],
        ).select(pl.lit(name).alias("label"), pl.col("*"))

    def find_bordering(self, id: str):
        return self.query(
            f"""\
SELECT (?bordering as ?borderingUri) ?borderingLabel WHERE {{
    ?bordering wdt:P31 wd:Q3624078. # 'instance of' 'sovereign state'
    ?bordering wdt:P47 wd:{id}. # 'shares border with'
    FILTER(NOT EXISTS {{ ?bordering (p:P31/ps:P31) wd:Q3024240. }}) # is NOT a 'historical country'
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}""",
            schema=["borderingUri", "borderingLabel"],
        ).select(pl.lit(id).alias("id"), pl.col("*"))


wikidata = WikidataRequests()

In [4]:
def id_from_uri(uri: str):
    # http://www.wikidata.org/entity/Q39 -> Q39
    return uri.rsplit("/", 1)[1]

In [5]:
input_countries = (
    pl.scan_csv("./data/landlocked 2023-06-26 - Лист1.csv")
    .select(pl.col("landlocked country").alias("label"))
    .collect()
    .to_series()
    .to_list()
)
input_countries

['Switzerland',
 'Austria',
 'Czech Republic',
 'Hungary',
 'Serbia',
 'Slovakia',
 'Ethiopia',
 'Kazakhstan',
 'Luxembourg',
 'Belarus',
 'Uganda',
 'Nepal',
 'Azerbaijan',
 'Armenia',
 'North Macedonia',
 'Uzbekistan',
 'Zimbabwe',
 'Malawi',
 'Zambia',
 'Moldova',
 'Botswana',
 'Burkina Faso',
 'Rwanda',
 'Mongolia',
 'Bolivia',
 'Paraguay',
 'Mali',
 'Kyrgyzstan',
 'Laos',
 'Afghanistan',
 'Niger',
 'Liechtenstein',
 'Tajikistan',
 'Eswatini',
 'Bhutan',
 'Burundi',
 'Lesotho',
 'Chad',
 'Central African Republic',
 'Vatican City',
 'South Sudan',
 'San Marino',
 'Andorra',
 'Turkmenistan']

In [6]:
def find_countries(countries):
    dfs = []
    for country in tqdm(countries, total=len(countries)):
        dfs.append(wikidata.find_country(country))
    return pl.DataFrame({"label": countries}).join(
        pl.concat(dfs), on="label", how="left"
    )

In [7]:
country_codes = find_countries(input_countries)
print(
    "Found",
    len(country_codes.filter(pl.col("uri").is_not_null())),
    "/",
    len(country_codes),
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:16<00:00,  2.70it/s]

Found 44 / 44


In [13]:
country_codes = country_codes.with_columns(pl.col("uri").apply(id_from_uri).alias("id"))
country_codes

label,uri,id
str,str,str
"""Switzerland""","""http://www.wikidata.org/entity/Q39""","""Q39"""
"""Austria""","""http://www.wikidata.org/entity/Q40""","""Q40"""
"""Czech Republic""","""http://www.wikidata.org/entity/Q213""","""Q213"""
"""Hungary""","""http://www.wikidata.org/entity/Q28""","""Q28"""
"""Serbia""","""http://www.wikidata.org/entity/Q403""","""Q403"""
"""Slovakia""","""http://www.wikidata.org/entity/Q214""","""Q214"""
"""Ethiopia""","""http://www.wikidata.org/entity/Q115""","""Q115"""
"""Kazakhstan""","""http://www.wikidata.org/entity/Q232""","""Q232"""
"""Luxembourg""","""http://www.wikidata.org/entity/Q32""","""Q32"""


In [14]:
def find_bordering_all(ids: list[str]):
    dfs = []
    for id in tqdm(ids, total=len(ids)):
        dfs.append(wikidata.find_bordering(id))
    return pl.concat(dfs)

In [15]:
bordering = find_bordering_all(country_codes["id"].to_list())
bordering

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:16<00:00,  2.72it/s]


id,borderingLabel,borderingUri
str,str,str
"""Q39""","""Italy""","""http://www.wikidata.org/entity/Q38"""
"""Q39""","""Austria""","""http://www.wikidata.org/entity/Q40"""
"""Q39""","""France""","""http://www.wikidata.org/entity/Q142"""
"""Q39""","""Germany""","""http://www.wikidata.org/entity/Q183"""
"""Q39""","""Liechtenstein""","""http://www.wikidata.org/entity/Q347"""
"""Q40""","""Hungary""","""http://www.wikidata.org/entity/Q28"""
"""Q40""","""Italy""","""http://www.wikidata.org/entity/Q38"""
"""Q40""","""Switzerland""","""http://www.wikidata.org/entity/Q39"""
"""Q40""","""Germany""","""http://www.wikidata.org/entity/Q183"""


In [18]:
final = country_codes.join(bordering, on="id").select(
    [
        pl.col("label"),
        pl.col("id"),
        pl.col("borderingLabel"),
        pl.col("borderingUri").apply(id_from_uri).alias("borderingId"),
    ]
)
final

label,id,borderingLabel,borderingId
str,str,str,str
"""Switzerland""","""Q39""","""Italy""","""Q38"""
"""Switzerland""","""Q39""","""Austria""","""Q40"""
"""Switzerland""","""Q39""","""France""","""Q142"""
"""Switzerland""","""Q39""","""Germany""","""Q183"""
"""Switzerland""","""Q39""","""Liechtenstein""","""Q347"""
"""Austria""","""Q40""","""Hungary""","""Q28"""
"""Austria""","""Q40""","""Italy""","""Q38"""
"""Austria""","""Q40""","""Switzerland""","""Q39"""
"""Austria""","""Q40""","""Germany""","""Q183"""


In [19]:
final.write_csv("./data/bordering.csv")